In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.5 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# Replaced 0 with nan and then nan imputed with mean

In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters/ Search space
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


# TPE Sampler

In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-02-23 12:45:09,300] A new study created in memory with name: no-name-1f6ebdd4-4ea0-40c2-a79d-0a1f5de9aedf
[I 2025-02-23 12:45:12,485] Trial 0 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 150, 'max_depth': 9}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-02-23 12:45:14,974] Trial 1 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 126, 'max_depth': 5}. Best is trial 1 with value: 0.7690875232774674.
[I 2025-02-23 12:45:16,222] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 97, 'max_depth': 7}. Best is trial 1 with value: 0.7690875232774674.
[I 2025-02-23 12:45:17,889] Trial 3 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 114, 'max_depth': 14}. Best is trial 1 with value: 0.7690875232774674.
[I 2025-02-23 12:45:19,041] Trial 4 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 167, 'max_depth': 4}. Best is trial 1 with value: 0.769087523

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 198, 'max_depth': 16}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


# Random Sampler
# increase our accuracy so direction="maximise"

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

# Best value of Hyperparameter

In [11]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 186, 'max_depth': 16}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


# Grid Search Sampler

In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [ ]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [ ]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12353185368595372, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.763501,2025-02-23 12:47:08.710413,2025-02-23 12:47:10.226694,0 days 00:00:01.516281,NaN,True,RandomForest,NaN,NaN,NaN,13.0,4.0,5.0,278.0,COMPLETE
1,1,0.761639,2025-02-23 12:47:10.227946,2025-02-23 12:47:11.260194,0 days 00:00:01.032248,NaN,True,RandomForest,NaN,NaN,NaN,18.0,9.0,2.0,193.0,COMPLETE
2,2,0.780261,2025-02-23 12:47:11.261436,2025-02-23 12:47:11.904477,0 days 00:00:00.643041,NaN,NaN,GradientBoosting,NaN,NaN,0.029880,6.0,8.0,6.0,64.0,COMPLETE
3,3,0.774674,2025-02-23 12:47:11.905795,2025-02-23 12:47:12.498809,0 days 00:00:00.593014,NaN,False,RandomForest,NaN,NaN,NaN,5.0,6.0,8.0,144.0,COMPLETE
4,4,0.754190,2025-02-23 12:47:12.500092,2025-02-23 12:47:14.020663,0 days 00:00:01.520571,NaN,NaN,GradientBoosting,NaN,NaN,0.193714,5.0,1.0,4.0,173.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.787709,2025-02-23 12:47:50.154675,2025-02-23 12:47:50.187199,0 days 00:00:00.032524,0.100601,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.765363,2025-02-23 12:47:50.188395,2025-02-23 12:47:50.230327,0 days 00:00:00.041932,0.309509,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.772812,2025-02-23 12:47:50.231483,2025-02-23 12:47:51.302272,0 days 00:00:01.070789,NaN,False,RandomForest,NaN,NaN,NaN,18.0,8.0,9.0,241.0,COMPLETE
98,98,0.770950,2025-02-23 12:47:51.303588,2025-02-23 12:47:51.351076,0 days 00:00:00.047488,0.153100,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,77
RandomForest,12
GradientBoosting,11


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.749111
RandomForest,0.766139
SVM,0.776754


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [34]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 3.1 MB/s eta 0:00:00


In [35]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-02-23 12:57:52,192] A new study created in memory with name: no-name-4b52c323-5ccb-4a8e-94b3-07cb1d810ef3


[0]	train-mlogloss:0.86374	eval-mlogloss:0.86307
[1]	train-mlogloss:0.65112	eval-mlogloss:0.63828
[2]	train-mlogloss:0.50379	eval-mlogloss:0.48898
[3]	train-mlogloss:0.40143	eval-mlogloss:0.37966
[4]	train-mlogloss:0.32685	eval-mlogloss:0.29984
[5]	train-mlogloss:0.26681	eval-mlogloss:0.23302
[6]	train-mlogloss:0.22341	eval-mlogloss:0.18713
[7]	train-mlogloss:0.19398	eval-mlogloss:0.15189
[8]	train-mlogloss:0.16835	eval-mlogloss:0.12339
[9]	train-mlogloss:0.14861	eval-mlogloss:0.10318
[10]	train-mlogloss:0.13400	eval-mlogloss:0.09117
[11]	train-mlogloss:0.12467	eval-mlogloss:0.08032
[12]	train-mlogloss:0.12016	eval-mlogloss:0.07607
[13]	train-mlogloss:0.11352	eval-mlogloss:0.06690
[14]	train-mlogloss:0.11104	eval-mlogloss:0.06396
[15]	train-mlogloss:0.10941	eval-mlogloss:0.06123
[16]	train-mlogloss:0.10761	eval-mlogloss:0.05900
[17]	train-mlogloss:0.10526	eval-mlogloss:0.05620
[18]	train-mlogloss:0.10428	eval-mlogloss:0.05417
[19]	train-mlogloss:0.10423	eval-mlogloss:0.05418
[20]	train

[I 2025-02-23 12:57:54,938] Trial 0 finished with value: 1.0 and parameters: {'lambda': 7.686458865042928e-08, 'alpha': 1.2738560900825797e-08, 'eta': 0.25214054437858696, 'gamma': 0.5405970137060753, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.8569370604218918, 'colsample_bytree': 0.7083552227955532}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:1.06142	eval-mlogloss:1.06045
[1]	train-mlogloss:1.01406	eval-mlogloss:1.01126
[2]	train-mlogloss:0.96888	eval-mlogloss:0.96374
[3]	train-mlogloss:0.92510	eval-mlogloss:0.91916
[4]	train-mlogloss:0.88542	eval-mlogloss:0.87764
[5]	train-mlogloss:0.84607	eval-mlogloss:0.83634
[6]	train-mlogloss:0.81059	eval-mlogloss:0.80076
[7]	train-mlogloss:0.77862	eval-mlogloss:0.76777
[8]	train-mlogloss:0.74782	eval-mlogloss:0.73434
[9]	train-mlogloss:0.71778	eval-mlogloss:0.70361
[10]	train-mlogloss:0.69019	eval-mlogloss:0.67475
[11]	train-mlogloss:0.66378	eval-mlogloss:0.64617
[12]	train-mlogloss:0.64624	eval-mlogloss:0.62995
[13]	train-mlogloss:0.62213	eval-mlogloss:0.60484
[14]	train-mlogloss:0.60398	eval-mlogloss:0.58787
[15]	train-mlogloss:0.58272	eval-mlogloss:0.56513
[16]	train-mlogloss:0.56282	eval-mlogloss:0.54432
[17]	train-mlogloss:0.54460	eval-mlogloss:0.52743
[18]	train-mlogloss:0.52501	eval-mlogloss:0.50654
[19]	train-mlogloss:0.50828	eval-mlogloss:0.49342
[20]	train

[I 2025-02-23 12:57:57,816] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.0061897131758855695, 'alpha': 8.319184939677234e-07, 'eta': 0.03878243960898765, 'gamma': 2.0854866624644816e-05, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.5980956231184185, 'colsample_bytree': 0.6774846275186113}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.06987	eval-mlogloss:1.06900
[1]	train-mlogloss:1.02828	eval-mlogloss:1.02547
[2]	train-mlogloss:0.98868	eval-mlogloss:0.98341
[3]	train-mlogloss:0.94969	eval-mlogloss:0.94356


[I 2025-02-23 12:58:30,172] Trial 48 pruned. Trial was pruned at iteration 4.


[0]	train-mlogloss:0.87552	eval-mlogloss:0.87693


[I 2025-02-23 12:58:31,333] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 7.686458865042928e-08, 'alpha': 1.2738560900825797e-08, 'eta': 0.25214054437858696, 'gamma': 0.5405970137060753, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.8569370604218918, 'colsample_bytree': 0.7083552227955532}
Best accuracy: 1.0


In [36]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()